In [1]:
#!/usr/bin/env PYTHONHASHSEED=1234 python3

# Copyright 2014-2019 Brett Slatkin, Pearson Education Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reproduce book environment
import random
random.seed(1234)

import logging
from pprint import pprint
from sys import stdout as STDOUT

# Write all output to a temporary directory
import atexit
import gc
import io
import os
import tempfile

TEST_DIR = tempfile.TemporaryDirectory()
atexit.register(TEST_DIR.cleanup)

# Make sure Windows processes exit cleanly
OLD_CWD = os.getcwd()
atexit.register(lambda: os.chdir(OLD_CWD))
os.chdir(TEST_DIR.name)

def close_open_files():
    everything = gc.get_objects()
    for obj in everything:
        if isinstance(obj, io.IOBase):
            obj.close()

atexit.register(close_open_files)

<function __main__.close_open_files()>

In [2]:
# Example 1
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        global print
        orig_print = print
        print(f'* Running {meta}.__new__ for {name}')
        print('Bases:', bases)
        print = pprint
        print(class_dict)
        print = orig_print
        return type.__new__(meta, name, bases, class_dict)

class MyClass(metaclass=Meta):
    stuff = 123

    def foo(self):
        pass

class MySubclass(MyClass):
    other = 567

    def bar(self):
        pass

* Running <class '__main__.Meta'>.__new__ for MyClass
Bases: ()
{'__module__': '__main__',
 '__qualname__': 'MyClass',
 'foo': <function MyClass.foo at 0x7cad44951310>,
 'stuff': 123}
* Running <class '__main__.Meta'>.__new__ for MySubclass
Bases: (<class '__main__.MyClass'>,)
{'__module__': '__main__',
 '__qualname__': 'MySubclass',
 'bar': <function MySubclass.bar at 0x7cad449721f0>,
 'other': 567}


In [3]:
# Example 2
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate subclasses of the Polygon class
        if bases:
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)

class Polygon(metaclass=ValidatePolygon):
    sides = None  # Must be specified by subclasses

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

class Triangle(Polygon):
    sides = 3

class Rectangle(Polygon):
    sides = 4

class Nonagon(Polygon):
    sides = 9

assert Triangle.interior_angles() == 180
assert Rectangle.interior_angles() == 360
assert Nonagon.interior_angles() == 1260

In [4]:
# Example 3
try:
    print('Before class')
    
    class Line(Polygon):
        print('Before sides')
        sides = 2
        print('After sides')
    
    print('After class')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-4-661126f6e9fd>", line 5, in <module>
    class Line(Polygon):
  File "<ipython-input-3-b71e304cdb59>", line 7, in __new__
    raise ValueError('Polygons need 3+ sides')
ValueError: Polygons need 3+ sides
Before class
Before sides
After sides


In [5]:
# Example 4
class BetterPolygon:
    sides = None  # Must be specified by subclasses

    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.sides < 3:
            raise ValueError('Polygons need 3+ sides')

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

class Hexagon(BetterPolygon):
    sides = 6

assert Hexagon.interior_angles() == 720

In [6]:
# Example 5
try:
    print('Before class')
    
    class Point(BetterPolygon):
        sides = 1
    
    print('After class')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-6-781f7803ddb1>", line 5, in <module>
    class Point(BetterPolygon):
  File "<ipython-input-5-3297e5db5606>", line 8, in __init_subclass__
    raise ValueError('Polygons need 3+ sides')
ValueError: Polygons need 3+ sides
Before class


In [7]:
# Example 6
class ValidateFilled(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate subclasses of the Filled class
        if bases:
            if class_dict['color'] not in ('red', 'green'):
                raise ValueError('Fill color must be supported')
        return type.__new__(meta, name, bases, class_dict)

class Filled(metaclass=ValidateFilled):
    color = None  # Must be specified by subclasses

In [8]:
# Example 7
try:
    class RedPentagon(Filled, Polygon):
        color = 'blue'
        sides = 5
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-8-01319cebde6f>", line 3, in <module>
    class RedPentagon(Filled, Polygon):
TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases


In [9]:
# Example 8
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate non-root classes
        if not class_dict.get('is_root'):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)

class Polygon(metaclass=ValidatePolygon):
    is_root = True
    sides = None  # Must be specified by subclasses

class ValidateFilledPolygon(ValidatePolygon):
    def __new__(meta, name, bases, class_dict):
        # Only validate non-root classes
        if not class_dict.get('is_root'):
            if class_dict['color'] not in ('red', 'green'):
                raise ValueError('Fill color must be supported')
        return super().__new__(meta, name, bases, class_dict)

class FilledPolygon(Polygon, metaclass=ValidateFilledPolygon):
    is_root = True
    color = None  # Must be specified by subclasses

In [10]:
# Example 9
class GreenPentagon(FilledPolygon):
    color = 'green'
    sides = 5

greenie = GreenPentagon()
assert isinstance(greenie, Polygon)

In [11]:
# Example 10
try:
    class OrangePentagon(FilledPolygon):
        color = 'orange'
        sides = 5
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-11-8ab8ab521e2b>", line 3, in <module>
    class OrangePentagon(FilledPolygon):
  File "<ipython-input-9-2e5564fc70e4>", line 19, in __new__
    raise ValueError('Fill color must be supported')
ValueError: Fill color must be supported


In [12]:
# Example 11
try:
    class RedLine(FilledPolygon):
        color = 'red'
        sides = 2
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-12-32a7c489e92e>", line 3, in <module>
    class RedLine(FilledPolygon):
  File "<ipython-input-9-2e5564fc70e4>", line 20, in __new__
    return super().__new__(meta, name, bases, class_dict)
  File "<ipython-input-9-2e5564fc70e4>", line 7, in __new__
    raise ValueError('Polygons need 3+ sides')
ValueError: Polygons need 3+ sides


In [13]:
# Example 12
class Filled:
    color = None  # Must be specified by subclasses

    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.color not in ('red', 'green', 'blue'):
            raise ValueError('Fills need a valid color')

In [14]:
# Example 13
class RedTriangle(Filled, BetterPolygon):
    color = 'red'
    sides = 3

ruddy = RedTriangle()
assert isinstance(ruddy, Filled)
assert isinstance(ruddy, BetterPolygon)

In [15]:
# Example 14
try:
    print('Before class')
    
    class BlueLine(Filled, BetterPolygon):
        color = 'blue'
        sides = 2
    
    print('After class')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-15-f6d26b60aac0>", line 5, in <module>
    class BlueLine(Filled, BetterPolygon):
  File "<ipython-input-13-75d0265a7a15>", line 6, in __init_subclass__
    super().__init_subclass__()
  File "<ipython-input-5-3297e5db5606>", line 8, in __init_subclass__
    raise ValueError('Polygons need 3+ sides')
ValueError: Polygons need 3+ sides
Before class


In [16]:
# Example 15
try:
    print('Before class')
    
    class BeigeSquare(Filled, BetterPolygon):
        color = 'beige'
        sides = 4
    
    print('After class')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-16-116618672645>", line 5, in <module>
    class BeigeSquare(Filled, BetterPolygon):
  File "<ipython-input-13-75d0265a7a15>", line 8, in __init_subclass__
    raise ValueError('Fills need a valid color')
ValueError: Fills need a valid color
Before class


In [17]:
# Example 16
class Top:
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Top for {cls}')

class Left(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Left for {cls}')

class Right(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Right for {cls}')

class Bottom(Left, Right):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Bottom for {cls}')

Top for <class '__main__.Left'>
Top for <class '__main__.Right'>
Top for <class '__main__.Bottom'>
Right for <class '__main__.Bottom'>
Left for <class '__main__.Bottom'>
